## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-08-16-59-34 +0000,bbc,Saudi-led coalition says UAE helped smuggle se...,https://www.bbc.com/news/articles/c5y2mvp3r2do...
1,2026-01-08-16-59-04 +0000,nypost,Wealth tax threat prompts at least six billion...,https://nypost.com/2026/01/08/business/wealth-...
2,2026-01-08-16-56-11 +0000,wapo,Trump appears to back away from threats to Col...,https://www.washingtonpost.com/world/2026/01/0...
3,2026-01-08-16-53-33 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...
4,2026-01-08-16-53-06 +0000,nypost,ICE agent who opened fire on Renee Good was dr...,https://nypost.com/2026/01/08/us-news/ice-agen...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,79
55,venezuela,36
31,ice,32
32,shooting,29
33,minneapolis,24


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2026-01-08-16-53-33 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...,181
122,2026-01-08-11-54-03 +0000,nypost,President Trump’s emotional reaction to watchi...,https://nypost.com/2026/01/08/us-news/presiden...,175
58,2026-01-08-15-01-16 +0000,nyt,Videos Contradict Trump Administration Account...,https://www.nytimes.com/video/us/1000000106310...,175
259,2026-01-08-00-40-05 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...,167
201,2026-01-08-06-43-53 +0000,nyt,Minnesota’s Dispute With Trump Administration ...,https://www.nytimes.com/2026/01/07/us/minnesot...,158


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,181,2026-01-08-16-53-33 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...
346,91,2026-01-07-20-31-48 +0000,bbc,US will control Venezuela oil sales 'indefinit...,https://www.bbc.com/news/articles/ckgn7p7g79wo...
251,64,2026-01-08-01-34-00 +0000,wsj,Trump Administration Draws Up New Legal Justif...,https://www.wsj.com/politics/policy/trump-admi...
4,60,2026-01-08-16-53-06 +0000,nypost,ICE agent who opened fire on Renee Good was dr...,https://nypost.com/2026/01/08/us-news/ice-agen...
96,54,2026-01-08-13-00-00 +0000,wsj,The U.S. pumps more oil than any other country...,https://www.wsj.com/business/energy-oil/us-ven...
1,44,2026-01-08-16-59-04 +0000,nypost,Wealth tax threat prompts at least six billion...,https://nypost.com/2026/01/08/business/wealth-...
216,40,2026-01-08-04-25-00 +0000,wsj,"Rep. Steny Hoyer of Maryland, a longtime membe...",https://www.wsj.com/politics/policy/steny-hoye...
87,39,2026-01-08-13-34-21 +0000,cbc,"Child killed in north Gaza by Israeli fire, fa...",https://www.cbc.ca/news/world/child-killed-nor...
110,38,2026-01-08-12-03-15 +0000,nypost,Bondi Beach hero Ahmed Al-Ahmed honored at NYC...,https://nypost.com/2026/01/08/us-news/bondi-be...
111,38,2026-01-08-12-01-00 +0000,wsj,China Warns AI Startups Seeking to Emulate Met...,https://www.wsj.com/tech/china-warns-ai-startu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
